In [50]:
import os

In [51]:
%pwd

'd:\\Repositories\\ML-geo'

In [52]:
os.chdir("d:/Repositories/ML-geo/")

In [53]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/FBrownp/ML-geo.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="FBrownp"
os.environ["MLFLOW_TRACKING_PASSWORD"]="cceabdf6d4f5adc126b6cb03cc7cc4bf568e2591"

In [54]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    model_name_1: str
    model_name_2: str
    target_column_1: str 
    target_column_2: str 
    mlflow_uri: str
    all_params: dict
    transformation_path: Path



In [55]:
from XGboost_for_slopes.constants import *
from XGboost_for_slopes.utils.common import read_yaml, create_directories

In [56]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.Xgboost
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path  = config.test_data_path,
            model_path = config.model_path,
            model_name_1 = config.model_name_1,
            model_name_2 = config.model_name_2,
            target_column_1= schema.target_1,
            target_column_2= schema.target_2,
            mlflow_uri= "https://dagshub.com/FBrownp/ML-geo.mlflow",
            all_params= params,
            transformation_path= config.transformation_path
            )
        return model_evaluation_config
    

In [57]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error
import os 
from XGboost_for_slopes import logger
import pandas as pd
import xgboost as xgb
import joblib
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score
from XGboost_for_slopes.utils.common import save_json
import mlflow
import mlflow.xgboost
from  urllib.parse import urlparse

In [58]:
class ModelEvaluation():
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def get_model_evaluation_object(self):
        
        test_data_df      = pd.read_csv(self.config.test_data_path)
        
        Transform_pipeline = joblib.load(self.config.transformation_path)


        y_data_test_1 = test_data_df[self.config.target_column_1]
        X_data_test_1 = test_data_df.drop([self.config.target_column_1, self.config.target_column_2], axis = 1)
        X_data_test_1  = Transform_pipeline.fit_transform(X_data_test_1)


        y_data_test_2 = test_data_df[self.config.target_column_2]
        X_data_test_2 = test_data_df.drop([self.config.target_column_1, self.config.target_column_2], axis = 1)
        X_data_test_2  = Transform_pipeline.fit_transform(X_data_test_2)


        model_1 = joblib.load(os.path.join(self.config.model_path,self.config.model_name_1))
        model_2 = joblib.load(os.path.join(self.config.model_path,self.config.model_name_2))


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            r2_value_model_1 = r2_score(y_data_test_1, model_1.predict(X_data_test_1))
            MSE_value_model_1 = mean_squared_error(y_data_test_1, model_1.predict(X_data_test_1))
            MAPE_value_model_1 = mean_absolute_percentage_error(y_data_test_1, model_1.predict(X_data_test_1))


            r2_value_model_2 = r2_score(y_data_test_2, model_2.predict(X_data_test_2))
            MSE_value_model_2 = mean_squared_error(y_data_test_2, model_2.predict(X_data_test_2))
            MAPE_value_model_2 = mean_absolute_percentage_error(y_data_test_2, model_2.predict(X_data_test_2))

            scores = {"r2_m1" : r2_value_model_1,
                    "MSE_m1": MSE_value_model_1,
                    "MAPE_m1": MAPE_value_model_1,
                    "r2_m2" : r2_value_model_2,
                    "MSE_m2": MSE_value_model_2,
                    "MAPE_m2": MAPE_value_model_2
                    }

            save_json(path= Path(os.path.join(self.config.root_dir,"scores.json")), data = scores)
            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("r2_m1",r2_value_model_1)
            mlflow.log_metric("r2_m2",r2_value_model_2)
            mlflow.log_metric("MSE_m1",MSE_value_model_1)
            mlflow.log_metric("MSE_m2",MSE_value_model_2)
            mlflow.log_metric("MAPE_m1",MAPE_value_model_1)
            mlflow.log_metric("MAPE_m2",MAPE_value_model_2)

            if tracking_url_type_store != "file":
                mlflow.xgboost.log_model(model_1, "model_1", registered_model_name="XGboost_FSs")
                mlflow.xgboost.log_model(model_2, "model_2", registered_model_name="XGboost_FSgmp")
            else:
                mlflow.xgboost.log_model(model_1, "model_1")



In [59]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config = model_evaluation_config)
    model_evaluation.get_model_evaluation_object()

except Exception as e:
    raise e

[2023-12-07 17:01:26,117: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-07 17:01:26,119: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-07 17:01:26,121: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-07 17:01:26,123: INFO: common: created directory at: artifacts]
[2023-12-07 17:01:26,124: INFO: common: created directory at: artifacts/model_evaluation]


[2023-12-07 17:01:26,704: INFO: common: json file saved at: artifacts\model_evaluation\scores.json]


d:\Repositories\ML-geo\venv\Lib\site-packages\xgboost\core.py:160: UserWarning: [17:01:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
Registered model 'XGboost_FSs' already exists. Creating a new version of this model...
2023/12/07 17:01:35 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: XGboost_FSs, version 3
Created version '3' of model 'XGboost_FSs'.
d:\Repositories\ML-geo\venv\Lib\site-packages\xgboost\core.py:160: UserWarning: [17:01:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecat